In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/train.parquet')
df['hour'] = df.ts % (60*60*24)

df = df.loc[ (df.hour >= 60*60*14)|(df.hour < 60*60*2) ]

df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(92113133, 6)


,session,aid,ts,type,hour,day
0,0,1517085,1659304800,0,79200,338400
1,0,1563459,1659304904,0,79304,338504
2,0,1309446,1659367439,0,55439,401039
3,0,16246,1659367719,0,55719,401319
4,0,1781822,1659367871,0,55871,401471


In [3]:
df = df.sort_values(['session','ts'])
df['prev'] = df.groupby('session').aid.diff(1)
df['next'] = df.groupby('session').aid.diff(-1)

df['order'] = (df.type==2).astype('int8')
df['cart'] = (df.type==1).astype('int8')

In [4]:
df.head()

,session,aid,ts,type,hour,day,prev,next,order,cart
0,0,1517085,1659304800,0,79200,338400,<NA>,-46374,0,0
1,0,1563459,1659304904,0,79304,338504,46374,254013,0,0
2,0,1309446,1659367439,0,55439,401039,-254013,1293200,0,0
3,0,16246,1659367719,0,55719,401319,-1293200,-1765576,0,0
4,0,1781822,1659367871,0,55871,401471,1765576,629148,0,0


In [5]:
df['n'] = df.groupby(['session','aid','type']).aid.transform('count')
tmp = df.loc[df['type']==2].drop_duplicates(['session','aid']).groupby('aid').n.mean()
tmp2 = df.loc[df['type']==1].drop_duplicates(['session','aid']).groupby('aid').n.mean()

In [6]:
tmp.head()

aid
330251      1.0
1300968     2.0
694554      1.0
753208     <NA>
44673       1.0
Name: n, dtype: float64

In [7]:
tmp.max(), tmp2.max()

(172.0, 255.0)

In [8]:
item_features3 = df.groupby('aid').agg({'prev':'nunique','next':'nunique','order':'sum','cart':'sum'})
item_features3 = item_features3.merge(tmp,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','buy_count3']
item_features3 = item_features3.merge(tmp2,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','order_repeat3','cart_repeat3']
item_features3 = item_features3.fillna(-1)

i32 = ['prev3','next3','orders3','carts3']
for c in i32: item_features3[c] = item_features3[c].astype('int32')
    
f32 = ['order_repeat3','cart_repeat3']
for c in f32: item_features3[c] = item_features3[c].astype('float32')
    
del tmp, tmp2
gc.collect()

64

In [9]:
item_features3.head()

,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,
15394,12,10,1,1,-1.0,-1.0
15406,10,9,3,4,3.0,1.0
15400,1,1,0,1,-1.0,1.0
15401,21,22,0,3,-1.0,1.5
15408,19,25,0,4,-1.0,6.0


In [10]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio3','count_item3','count_user3','hour_mean3','day_mean3']

In [11]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3
aid,,,,,
0,0.000000,19,14,67888.947368,454415.263158
1,0.058824,17,17,64961.117647,308914.058824
2,0.000000,2,2,59939.500000,275939.500000
3,0.096367,633,359,62304.916272,321641.409163
4,0.030928,97,67,64757.804124,301689.762887


In [12]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std3','hour_std3','day_std3']

f32 = ['buy_ratio_std3','hour_std3','day_std3']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [13]:
item_features2.head()

,buy_ratio_std3,hour_std3,day_std3
aid,,,
1557217,0.342803,15490.552734,174911.468750
1701136,0.786796,6037.943848,135858.109375
1612771,0.000000,8422.773438,65652.023438
1255532,0.208321,8689.463867,153842.375000
806983,0.000000,27193.720703,189931.234375


In [14]:
item_features['repeat3'] = item_features.count_item3 / item_features.count_user3

f32 = ['buy_ratio3','hour_mean3','day_mean3','repeat3']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item3','count_user3']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [15]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [16]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,,,,,,,,,,
0,0.000000,19,14,67888.945312,454415.25000,1.357143,0.000000,9168.577148,130724.406250,14,16,0,0,-1.0,-1.000000
1,0.058824,17,17,64961.117188,308914.06250,1.000000,0.242536,8883.181641,160759.859375,16,14,0,1,-1.0,-1.000000
2,0.000000,2,2,59939.500000,275939.50000,1.000000,0.000000,11249.361328,72343.390625,2,2,0,0,-1.0,-1.000000
3,0.096367,633,359,62304.917969,321641.40625,1.763231,0.344766,11903.668945,169089.203125,112,115,10,41,1.5,2.416667
4,0.030928,97,67,64757.804688,301689.75000,1.447761,0.174022,8664.611328,179179.000000,62,59,0,3,-1.0,1.000000


In [17]:
item_features.dtypes

buy_ratio3        float32
count_item3         int32
count_user3         int32
hour_mean3        float32
day_mean3         float32
repeat3           float32
buy_ratio_std3    float32
hour_std3         float32
day_std3          float32
prev3               int32
next3               int32
orders3             int32
carts3              int32
order_repeat3     float32
cart_repeat3      float32
dtype: object

In [18]:
COLS = [f'{x[:-1]}8' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio8', 'count_item8', 'count_user8', 'hour_mean8', 'day_mean8', 'repeat8', 'buy_ratio_std8', 'hour_std8', 'day_std8', 'prev8', 'next8', 'orders8', 'carts8', 'order_repeat8', 'cart_repeat8']


In [19]:
item_features.to_parquet('../../data/item_user_features/item21.pqt')